In [ ]:
# Copyright (C) 2016 Delaters
# This file is part of PARTERRA_GEE.
#
# PARTERRA_GEE is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# PARTERRA_GEE is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
#  You should have received a copy of the GNU General Public License
#  along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [ ]:
import shapely.wkt
import os
from osgeo import ogr

from osgeo import gdal

ogr.UseExceptions()

# gdal.SetConfigOption('CPL_DEBUG', 'ON')
gdal.SetConfigOption('CPL_DEBUG', 'OFF')

use gdal_auth.py to get token, see http://www.gdal.org/drv_gft.html

In [3]:
# GFT_REFRESH_TOKEN = #os.environ['GFT_REFRESH_TOKEN']

In [4]:
def convert2ft(input_path, output_path, append=False, fix_geometry=False, simplify_geometry=False, start_index=0):
    filename = input_path

    src_ds = ogr.Open(input_path)
    src_lyr = src_ds.GetLayerByIndex(0)
    f = src_lyr.GetFeature(1)

    # create feature table
    dst_ds = ogr.GetDriverByName('GFT').Open('GFT:refresh=' + GFT_REFRESH_TOKEN, True)
    
    if append:
        dst_lyr = dst_ds.GetLayerByName(output_path)
    else:
        dst_lyr = dst_ds.CreateLayer(output_path)

        # create fields using OGR
        [dst_lyr.CreateField(f.GetFieldDefnRef(i)) for i in range(f.GetFieldCount())]

    index = 0
    batch_size = 250
    index_batch = 0
    for feat in src_lyr:
        if index < start_index:
            index = index + 1
            continue
        
        try:
            geom = shapely.wkt.loads(feat.GetGeometryRef().ExportToWkt())
        except Exception as e: 
            print('Error({0}), skipping geometry.'.format(e))
            continue

        if fix_geometry and not geom.is_valid:
            geom = geom.buffer(0.0)
        
        if simplify_geometry:
            geom = geom.simplify(0.004)

        f = ogr.Feature(dst_lyr.GetLayerDefn())
        
        # set field values
        for i in range(feat.GetFieldCount()):
            fd = feat.GetFieldDefnRef(i)
            f.SetField(fd.GetName(), feat.GetField(fd.GetName()))
            
            #print(fd.GetName())
            #print(feat.GetField(fd.GetName()))
        
        # set geometry    
        f.SetGeometry(ogr.CreateGeometryFromWkt(geom.to_wkt()))
        
        if index_batch == 0:
            dst_lyr.StartTransaction()
            
        index_batch = index_batch + 1
        
        # create feature
        feature = dst_lyr.CreateFeature(f)
        
        f.Destroy() 
        
        index = index + 1

        if index_batch > batch_size:
            dst_lyr.CommitTransaction()
            print('Inserted {0} features ...'.format(batch_size))

            index_batch = 0
        
    src_ds.Destroy()

    dst_ds.Destroy()

In [5]:
input_file = 'daressalaam-latest-lines.kml'
output_table = 'daressalaam-latest-lines-29-03-2016'

In [22]:
append = False
fix_geometry = True
simplify_geometry = False

# gdal.SetConfigOption('CPL_DEBUG', 'OFF')
    
convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry) 
# convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry, 10617) # append

Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...
Inserted 250 features ...


In [7]:
##### append = True
fix_geometry = True
simplify_geometry = False

# gdal.SetConfigOption('CPL_DEBUG', 'OFF')

ds = ogr.Open(input_file)
l = ds.GetLayerByIndex(0)
totalCount = l.GetFeatureCount()

def GetFeatureCount():
    driver = ogr.GetDriverByName('GFT')
    dataSource = driver.Open('GFT:refresh=' + GFT_REFRESH_TOKEN, True)
    layer = dataSource.GetLayer(12)

    return layer.GetFeatureCount()

count = GetFeatureCount()

print('{0} ({1:.2f}%)'.format(count, 100. * float(count) / totalCount))

while count < totalCount:
    try:
        # convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry) 
        convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry, count) # append
    except RuntimeError:
        count = GetFeatureCount()
        print('{0} ({1:.2f}%)'.format(count, 100. * float(count) / totalCount))

103968 (101.38%)


In [12]:
print(l.GetFeatureCount())

9420


In [6]:
driver = ogr.GetDriverByName('GFT')
dataSource = driver.Open('GFT:refresh=' + GFT_REFRESH_TOKEN, True)
print(dataSource)


layerCount = dataSource.GetLayerCount()

print('Layer count: {0}'.format(layerCount))

for i in range(layerCount):
    layer = dataSource.GetLayer(i)
    print('{0}: {1}'.format(i, layer.GetName()))

RuntimeError: Fetching OAuth2 access code from auth code failed.